In [162]:
import datetime
import email
import imaplib
import mailbox
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import re
import glob
from numpy import nansum
from numpy import nanmean
import datetime
import functools
from datetime import datetime

In [169]:
import math
from scipy.optimize import brentq
Gamma=0.66

#----------------numerical solution--------------------------
# Tw + (es*(Tw))/𝛾 = Ta + ea/ 𝛾
# Ta air temperature 
# RH relative humidit
def TwNumerical(Ta,RH):
    SaturatedVPTa = 6.11* np.exp( (17.269*Ta) / (237.3+Ta) )
    VaporPressure = SaturatedVPTa*RH/100.0    
    def GetRoot(Tw):
        SaturatedVPTw = 6.11* np.exp( (17.269*Tw) / (237.3+Tw) )
        Function = Tw + SaturatedVPTw/Gamma-Ta-VaporPressure/Gamma
        return Function
    root,info = brentq(GetRoot,-20,50, full_output=True)
    return root


#----------------stull's formula----------------------
# use Stull's formula to calculate wet bulb temperature
# Ta air temperature 
# RH relative humidity
def Tw_Stull(Ta, RH):
    Tw_Stull=Ta*np.arctan(0.151977*(RH+8.313659)**0.5)+\
            np.arctan(Ta+RH)-np.arctan(RH-1.676331)+\
            np.arctan(0.023101*RH)*0.00391838*RH**(3./2.)-4.686035
    return Tw_Stull

def Clean_table_TW(df, st, tr):

    df['Lon'] = df['Lon'].str.replace('W','-').str.replace('E','').str.replace(' ','').astype(float)
    df['Lat'] = df['Lat'].str.replace('S','-').str.replace('N','').str.replace(' ','').astype(float)
    df['Temp'] = df['Temp'].str.replace('C','').str.replace(' ','').astype(float)
    df['Temp']=(df['Temp']-32)*5/9
    df['RH'] = df['RH'].str.replace('%','').str.replace(' ','').astype(float)
    df['Time']= pd.to_datetime(df['Time'])
    df['Biked']=np.nan
    df['Biked'][0]=(df['Time'][df.shape[0]-1]-df['Time'][0]).seconds/60
    df['Date']=np.nan
    df['Date'][0]=str('Trip '+tr+' ('+st[4:6]+'-'+st[-2:]+'-'+st[0:4]+')')
    df['Date_all']=str('Trip '+tr+' ('+st[4:6]+'-'+st[-2:]+'-'+st[0:4]+')')
    #df['Time']=df['Time'].replace(year=int(st[0:4]), month=int(st[4:6]), day=int(st[-2:]))
    df['Tw_Numerical']=df.apply(lambda x: TwNumerical(x['Temp'], x['RH']), axis=1)
    df['Tw_Stull']=Tw_Stull(df['Temp'], df['RH'])
    
    
    
    
    df = df.fillna('')
    return df

In [170]:
Dat_list=glob.glob('Bike study-6/*.txt')
Dat_list

['Bike study-6\\20191002-evening.txt',
 'Bike study-6\\20191002-morning.txt',
 'Bike study-6\\20191003-evening.txt',
 'Bike study-6\\20191003-morning.txt',
 'Bike study-6\\20191004-evening.txt',
 'Bike study-6\\20191004-morning.txt',
 'Bike study-6\\20191022-evening.txt',
 'Bike study-6\\20191022-morning.txt',
 'Bike study-6\\20191023-evening.txt',
 'Bike study-6\\20191023-morning.txt',
 'Bike study-6\\20191025-evening.txt',
 'Bike study-6\\20191025-morning.txt',
 'Bike study-6\\20191028-evening.txt',
 'Bike study-6\\20191028-morning.txt',
 'Bike study-6\\20191029-evening.txt',
 'Bike study-6\\20191029-morning.txt',
 'Bike study-6\\20191030-evening.txt',
 'Bike study-6\\20191030-morning.txt',
 'Bike study-6\\20191031-evening.txt',
 'Bike study-6\\20191031-morning .txt',
 'Bike study-6\\20191115.txt',
 'Bike study-6\\20191118.txt',
 'Bike study-6\\20191120-morning.txt',
 'Bike study-6\\20191120.txt',
 'Bike study-6\\20191122.txt',
 'Bike study-6\\20191125.txt',
 'Bike study-6\\20200224e

In [171]:
Dat_all=[]
ii=1
for item in Dat_list:
    Dat=pd.read_csv(item, names=["Time", "Lon", "Lat", "ID","Temp","RH"]) 
    Dat_clean=Clean_table_TW(Dat, str(item)[13:21], "{:0>2}".format(str(ii)))
    Dat_all.append(Dat_clean)
    ii=ii+1


C:\Users\tirth\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\tirth\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\tirth\Anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [150]:
(result['Time'][56]-result['Time'][55]).seconds

Time            datetime64[ns]
Lon                    float64
Lat                    float64
ID                      object
Temp                   float64
RH                     float64
Tw_Numerical           float64
Tw_Stull               float64
dtype: object

In [172]:
result= pd.concat(Dat_all, ignore_index=True)
result


,Time,Lon,Lat,ID,Temp,RH,Biked,Date,Date_all,Tw_Numerical,Tw_Stull
0,2020-08-20 08:08:12,-72.90217,41.35806,AFF9,23.500000,79.62,573.05,Trip 01 (10-02-2019),Trip 01 (10-02-2019),20.923797,20.838650
1,2020-08-20 08:08:13,-72.90217,41.35806,AFF9,23.527778,79.54,,,Trip 01 (10-02-2019),20.939260,20.854587
2,2020-08-20 08:08:14,-72.90216,41.35806,AFF9,23.527778,79.46,,,Trip 01 (10-02-2019),20.928626,20.843884
3,2020-08-20 08:08:15,-72.90216,41.35806,AFF9,23.500000,79.38,,,Trip 01 (10-02-2019),20.891918,20.806557
4,2020-08-20 08:08:16,-72.90217,41.35805,AFF9,23.527778,79.34,,,Trip 01 (10-02-2019),20.912667,20.827826
...,...,...,...,...,...,...,...,...,...,...,...
145288,2020-08-20 09:16:54,-72.92315,41.31662,AFF9,17.900000,53.31,,,Trip 53 (03-12-2020),12.501311,12.304962
145289,2020-08-20 09:16:55,-72.92315,41.31662,AFF9,17.900000,53.41,,,Trip 53 (03-12-2020),12.514033,12.317307
145290,2020-08-20 09:16:56,-72.92315,41.31662,AFF9,17.922222,53.56,,,Trip 53 (03-12-2020),12.551713,12.355246
145291,2020-08-20 09:16:58,-72.92315,41.31662,AFF9,17.911111,53.58,,,Trip 53 (03-12-2020),12.544953,12.347999


In [ ]:
# for index, row in result.iterrows():
#     kk=1
#     if ((result['Time'][index]-result[index-1][55]).seconds>=900):
#         result['Biked'][0]=(result['Time'][df.shape[0]-1]-result['Time'][0]).seconds/60
#         result['Date'][0]=str('Trip '+ "{:0>2}".format(str(kk))+' ('+st[4:6]+'-'+st[-2:]+'-'+st[0:4]+')')
#         result['Date_all']=str('Trip '+ "{:0>2}".format(str(kk))+' ('+st[4:6]+'-'+st[-2:]+'-'+st[0:4]+')')
        
    
#     else:
#         result['Biked'][index]=np.nan
#         result['Date'][index]=np.nan

In [98]:
result.to_csv('Compiled_Sara.csv')

In [90]:
ss=result.groupby('Date').max()
Fin=pd.DataFrame(ss.index)

Fin[Fin['Date']==""]=np.nan
Fin=Fin.dropna()
Fin

,Date
1,Trip 01 (10-02-2019)
2,Trip 02 (10-02-2019)
3,Trip 03 (10-03-2019)
4,Trip 04 (10-03-2019)
5,Trip 05 (10-04-2019)
6,Trip 06 (10-04-2019)
7,Trip 07 (10-22-2019)
8,Trip 08 (10-22-2019)
9,Trip 09 (10-23-2019)
10,Trip 10 (10-23-2019)


In [48]:
Fin.to_csv('Compiled_Sara_Date.csv')